In [1]:
## McKinsey Hackathon - submission by ShayanRay
## Model used: ETC2

In [2]:
# library imports
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn.svm import SVR
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
from sklearn import utils
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score

In [3]:
# Load Train and Test CSV

headerNames = ["ID","Gender","DOB","Lead_Creation_Date","City_Code","City_Category","Employer_Code","Employer_Category1",
               "Employer_Category2","Monthly_Income","Customer_Existing_Primary_Bank_Code","Primary_Bank_Type","Contacted",
               "Source","Source_Category","Existing_EMI","Loan_Amount","Loan_Period","Interest_Rate","EMI","Var1","Approved"]
prefix = "/data/McKinsey/"

# it is mentioned not to use ID for prediction 
# hence setting index_col = 0 takes care of removing ID field from dataset in both train and test dataframes.
traindf = pd.read_csv(prefix + "train.csv", header=None, delim_whitespace=False,  names=headerNames, index_col=0,) 
testdf = pd.read_csv(prefix + "test.csv", header=None, delim_whitespace=False,  names=headerNames, index_col=0,)
testdf = testdf.drop('Approved', axis=1)

#sample data for a quick run
#traindf = traindf.sample(frac=0.25, replace=True)

traindf.shape

(69713, 21)

In [4]:
#top 10 records displayed
traindf.head(10)

,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
ID,,,,,,,,,,,,,,,,,,,,,
APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,B001,...,N,S122,G,0.0,NaN,NaN,NaN,NaN,0,0
APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,B002,...,Y,S122,G,0.0,20000.0,2.0,13.25,953.0,10,0
APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,B003,...,Y,S143,B,0.0,45000.0,4.0,NaN,NaN,0,0
APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,B003,...,Y,S143,B,0.0,92000.0,5.0,NaN,NaN,7,0
APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,B001,...,Y,S134,B,2500.0,50000.0,2.0,NaN,NaN,10,0
APPI90914237819,Female,23/12/88,01/07/16,C10402,C,COM0007740,A,4.0,7000.0,B014,...,N,S133,B,0.0,NaN,NaN,NaN,NaN,0,0
APPK80327232033,Male,28/03/73,02/07/16,C10022,B,COM0030526,B,4.0,7500.0,B003,...,Y,S122,C,0.0,130000.0,5.0,14.85,3082.0,10,0
APPL20820172146,Female,12/11/90,03/07/16,C10003,A,COM0016738,A,4.0,3000.0,B006,...,Y,S133,B,0.0,30000.0,3.0,18.25,1088.0,0,0
APPM30977401211,Male,03/03/77,02/07/16,C10003,A,COM0010626,A,4.0,2500.0,B017,...,Y,S133,B,0.0,66000.0,5.0,20.00,1749.0,7,0


In [5]:
# Set of Unique Values for approved - it is a binary classification problem
print(traindf['Approved'].unique())
traindf.columns

[0 1]


Index(['Gender', 'DOB', 'Lead_Creation_Date', 'City_Code', 'City_Category',
       'Employer_Code', 'Employer_Category1', 'Employer_Category2',
       'Monthly_Income', 'Customer_Existing_Primary_Bank_Code',
       'Primary_Bank_Type', 'Contacted', 'Source', 'Source_Category',
       'Existing_EMI', 'Loan_Amount', 'Loan_Period', 'Interest_Rate', 'EMI',
       'Var1', 'Approved'],
      dtype='object')

In [6]:
# Train Data Stats
traindf.describe()


,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
count,65415.000000,6.971300e+04,69662.000000,42004.000000,42004.000000,22276.000000,22276.000000,69713.000000,69713.000000
mean,3.720187,5.622283e+03,360.928751,39429.982859,3.890629,19.213570,1101.466242,3.948446,0.014631
std,0.807374,1.747671e+05,2288.517927,30727.595990,1.167491,5.847136,752.661394,3.819214,0.120073
min,1.000000,0.000000e+00,0.000000,5000.000000,1.000000,11.990000,118.000000,0.000000,0.000000
25%,4.000000,1.650000e+03,0.000000,20000.000000,3.000000,15.250000,649.000000,0.000000,0.000000
50%,4.000000,2.500000e+03,0.000000,30000.000000,4.000000,18.000000,941.000000,2.000000,0.000000
75%,4.000000,4.000000e+03,350.000000,50000.000000,5.000000,20.000000,1295.000000,7.000000,0.000000
max,4.000000,3.838384e+07,545436.500000,300000.000000,6.000000,37.000000,13556.000000,10.000000,1.000000


In [7]:
# stats of categorical features
traindf.describe(include=['O'])

,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Customer_Existing_Primary_Bank_Code,Primary_Bank_Type,Contacted,Source,Source_Category
count,69713,69698,69713,68899,68899,65695,65695,60322,60322,69713,69713,69713
unique,2,10759,92,678,3,36617,3,57,2,2,29,7
top,Male,11/01/82,02/09/16,C10001,A,COM0000002,A,B001,P,Y,S122,B
freq,39949,253,1838,10007,49885,457,33336,14197,39619,45275,30941,29812


In [8]:
# for starters, fill every nan value with mean column values across the dataset.
# set to 0 for feature selection
#traindf = traindf.fillna(0.0) 
#testdf = testdf.fillna(0.0) 

#fill NaN values with mean for training
traindf['Employer_Category2'].fillna(traindf['Employer_Category2'].dropna().mean(), inplace=True)
traindf['Monthly_Income'].fillna(traindf['Monthly_Income'].dropna().mean(), inplace=True)
traindf['Existing_EMI'].fillna(traindf['Existing_EMI'].dropna().mean(), inplace=True)
traindf['Loan_Amount'].fillna(traindf['Loan_Amount'].dropna().mean(), inplace=True)
traindf['Loan_Period'].fillna(traindf['Loan_Period'].dropna().mean(), inplace=True)
traindf['Interest_Rate'].fillna(traindf['Interest_Rate'].dropna().mean(), inplace=True)
traindf['EMI'].fillna(traindf['EMI'].dropna().mean(), inplace=True)

#fill NaN values with mean for test
testdf['Employer_Category2'].fillna(testdf['Employer_Category2'].dropna().mean(), inplace=True)
testdf['Monthly_Income'].fillna(testdf['Monthly_Income'].dropna().mean(), inplace=True)
testdf['Existing_EMI'].fillna(testdf['Existing_EMI'].dropna().mean(), inplace=True)
testdf['Loan_Amount'].fillna(testdf['Loan_Amount'].dropna().mean(), inplace=True)
testdf['Loan_Period'].fillna(testdf['Loan_Period'].dropna().mean(), inplace=True)
testdf['Interest_Rate'].fillna(testdf['Interest_Rate'].dropna().mean(), inplace=True)
testdf['EMI'].fillna(testdf['EMI'].dropna().mean(), inplace=True)

traindf.head(10)

,Gender,DOB,Lead_Creation_Date,City_Code,City_Category,Employer_Code,Employer_Category1,Employer_Category2,Monthly_Income,Customer_Existing_Primary_Bank_Code,...,Contacted,Source,Source_Category,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Approved
ID,,,,,,,,,,,,,,,,,,,,,
APPC90493171225,Female,23/07/79,15/07/16,C10001,A,COM0044082,A,4.0,2000.0,B001,...,N,S122,G,0.0,39429.982859,3.890629,19.21357,1101.466242,0,0
APPD40611263344,Male,07/12/86,04/07/16,C10003,A,COM0000002,C,1.0,3500.0,B002,...,Y,S122,G,0.0,20000.000000,2.000000,13.25000,953.000000,10,0
APPE70289249423,Male,10/12/82,19/07/16,C10125,C,COM0005267,C,4.0,2250.0,B003,...,Y,S143,B,0.0,45000.000000,4.000000,19.21357,1101.466242,0,0
APPF80273865537,Male,30/01/89,09/07/16,C10477,C,COM0004143,A,4.0,3500.0,B003,...,Y,S143,B,0.0,92000.000000,5.000000,19.21357,1101.466242,7,0
APPG60994436641,Male,19/04/85,20/07/16,C10002,A,COM0001781,A,4.0,10000.0,B001,...,Y,S134,B,2500.0,50000.000000,2.000000,19.21357,1101.466242,10,0
APPI90914237819,Female,23/12/88,01/07/16,C10402,C,COM0007740,A,4.0,7000.0,B014,...,N,S133,B,0.0,39429.982859,3.890629,19.21357,1101.466242,0,0
APPK80327232033,Male,28/03/73,02/07/16,C10022,B,COM0030526,B,4.0,7500.0,B003,...,Y,S122,C,0.0,130000.000000,5.000000,14.85000,3082.000000,10,0
APPL20820172146,Female,12/11/90,03/07/16,C10003,A,COM0016738,A,4.0,3000.0,B006,...,Y,S133,B,0.0,30000.000000,3.000000,18.25000,1088.000000,0,0
APPM30977401211,Male,03/03/77,02/07/16,C10003,A,COM0010626,A,4.0,2500.0,B017,...,Y,S133,B,0.0,66000.000000,5.000000,20.00000,1749.000000,7,0


In [9]:
# Feature Engineering - Convert Categorical Data to Numeric > Gender
# convert Gender to numeric

#Train Data
traindf['Gender_numeric']  = 0.0 # default value
traindf.loc[traindf['Gender'] == 'Male', 'Gender_numeric'] = 1.0
traindf.loc[traindf['Gender'] == 'Female', 'Gender_numeric'] = 2.0
traindf = traindf.drop('Gender', axis=1)

#Test Data
testdf['Gender_numeric']  = 0.0 # default value
testdf.loc[testdf['Gender'] == 'Male', 'Gender_numeric'] = 1.0
testdf.loc[testdf['Gender'] == 'Female', 'Gender_numeric'] = 2.0
testdf = testdf.drop('Gender', axis=1)

In [10]:
# Feature Engineering - Convert Categorical Data to Numeric > City_Code
# convert City_Code to numeric

#Train Data
traindf['City_Code_numeric']  = 0.0 # default value
traindf.loc[traindf['City_Code'] != 0.0, 'City_Code_numeric'] = traindf['City_Code'].str.slice(start=1).astype(float)
traindf['City_Code_numeric'].fillna(traindf['City_Code_numeric'].dropna().mode()[0], inplace=True)
traindf = traindf.drop('City_Code', axis=1)

#Test Data

testdf['City_Code_numeric']  = 0.0 # default value
testdf.loc[testdf['City_Code'] != 0.0, 'City_Code_numeric'] = testdf['City_Code'].str.slice(start=1).astype(float)
testdf['City_Code_numeric'].fillna(testdf['City_Code_numeric'].dropna().mode()[0], inplace=True)
testdf = testdf.drop('City_Code', axis=1)

In [11]:
# Feature Engineering - Convert Categorical Data to Numeric > City_Category
# convert City_Category to numeric
traindf['City_Category'].unique()

#Train Data
traindf['City_Category_numeric']  = 0.0 # default value
traindf.loc[traindf['City_Category'] == 'A', 'City_Category_numeric'] = 1.0
traindf.loc[traindf['City_Category'] == 'B', 'City_Category_numeric'] = 2.0
traindf.loc[traindf['City_Category'] == 'C', 'City_Category_numeric'] = 3.0
traindf['City_Category_numeric'].unique()
traindf = traindf.drop('City_Category', axis=1)

#Test Data
testdf['City_Category_numeric']  = 0.0 # default value
testdf.loc[testdf['City_Category'] == 'A', 'City_Category_numeric'] = 1.0
testdf.loc[testdf['City_Category'] == 'B', 'City_Category_numeric'] = 2.0
testdf.loc[testdf['City_Category'] == 'C', 'City_Category_numeric'] = 3.0
testdf['City_Category_numeric'].unique()
testdf = testdf.drop('City_Category', axis=1)

In [12]:
# Feature Engineering - Convert Categorical Data to Numeric > Employer_Code
# convert Employer_Code to numeric
traindf['Employer_Code'].unique()


#Train Data
traindf['Employer_Code_numeric']  = 0.0 # default value
traindf.loc[traindf['Employer_Code'] != 0.0, 'Employer_Code_numeric'] = traindf['Employer_Code'].str.slice(start=3).astype(float)
traindf['Employer_Code_numeric'].fillna(traindf['Employer_Code_numeric'].dropna().mode()[0], inplace=True)
traindf = traindf.drop('Employer_Code', axis=1)

#Test Data

testdf['Employer_Code_numeric']  = 0.0 # default value
testdf.loc[testdf['Employer_Code'] != 0.0, 'Employer_Code_numeric'] = testdf['Employer_Code'].str.slice(start=3).astype(float)
testdf['Employer_Code_numeric'].fillna(testdf['Employer_Code_numeric'].dropna().mode()[0], inplace=True)
testdf = testdf.drop('Employer_Code', axis=1)

In [13]:
# Feature Engineering - Convert Categorical Data to Numeric > Employer_Category1
# convert Employer_Category1 to numeric
traindf['Employer_Category1'].unique()


#Train Data
traindf['Employer_Category1_numeric']  = 0.0 # default value
traindf.loc[traindf['Employer_Category1'] == 'A', 'Employer_Category1_numeric'] = 1.0
traindf.loc[traindf['Employer_Category1'] == 'B', 'Employer_Category1_numeric'] = 2.0
traindf.loc[traindf['Employer_Category1'] == 'C', 'Employer_Category1_numeric'] = 3.0
traindf = traindf.drop('Employer_Category1', axis=1)

#Test Data
testdf['Employer_Category1_numeric']  = 0.0 # default value
testdf.loc[testdf['Employer_Category1'] == 'A', 'Employer_Category1_numeric'] = 1.0
testdf.loc[testdf['Employer_Category1'] == 'B', 'Employer_Category1_numeric'] = 2.0
testdf.loc[testdf['Employer_Category1'] == 'C', 'Employer_Category1_numeric'] = 3.0
testdf = testdf.drop('Employer_Category1', axis=1)

In [14]:
# Feature Engineering - Convert Categorical Data to Numeric > Customer_Existing_Primary_Bank_Code
# convert Customer_Existing_Primary_Bank_Code to numeric
traindf['Customer_Existing_Primary_Bank_Code'].unique()

#Train Data
traindf['Customer_Existing_Primary_Bank_Code_numeric']  = 0.0 # default value
traindf.loc[traindf['Customer_Existing_Primary_Bank_Code'] != 0.0, 'Customer_Existing_Primary_Bank_Code_numeric'] = traindf['Customer_Existing_Primary_Bank_Code'].str.slice(start=1).astype(float)
traindf['Customer_Existing_Primary_Bank_Code_numeric'].fillna(traindf['Customer_Existing_Primary_Bank_Code_numeric'].dropna().mode()[0], inplace=True)
traindf = traindf.drop('Customer_Existing_Primary_Bank_Code', axis=1)

#Test Data

testdf['Customer_Existing_Primary_Bank_Code_numeric']  = 0.0 # default value
testdf.loc[testdf['Customer_Existing_Primary_Bank_Code'] != 0.0, 'Customer_Existing_Primary_Bank_Code_numeric'] = testdf['Customer_Existing_Primary_Bank_Code'].str.slice(start=1).astype(float)
testdf['Customer_Existing_Primary_Bank_Code_numeric'].fillna(testdf['Customer_Existing_Primary_Bank_Code_numeric'].dropna().mode()[0], inplace=True)
testdf = testdf.drop('Customer_Existing_Primary_Bank_Code', axis=1)

In [15]:
# Feature Engineering - Convert Categorical Data to Numeric > Primary_Bank_Type
# convert Primary_Bank_Type to numeric
traindf['Primary_Bank_Type'].unique()


#Train Data
traindf['Primary_Bank_Type_numeric']  = 0.0 # default value
traindf.loc[traindf['Primary_Bank_Type'] == 'P', 'Primary_Bank_Type_numeric'] = 1.0
traindf.loc[traindf['Primary_Bank_Type'] == 'G', 'Primary_Bank_Type_numeric'] = 2.0
traindf = traindf.drop('Primary_Bank_Type', axis=1)

#Test Data
testdf['Primary_Bank_Type_numeric']  = 0.0 # default value
testdf.loc[testdf['Primary_Bank_Type'] == 'P', 'Primary_Bank_Type_numeric'] = 1.0
testdf.loc[testdf['Primary_Bank_Type'] == 'G', 'Primary_Bank_Type_numeric'] = 2.0
testdf = testdf.drop('Primary_Bank_Type', axis=1)

In [16]:
# Feature Engineering - Convert Categorical Data to Numeric > Contacted
# convert Contacted to numeric
traindf['Contacted'].unique()


#Train Data
traindf['Contacted_numeric']  = 0.0 # default value
traindf.loc[traindf['Contacted'] == 'Y', 'Contacted_numeric'] = 1.0
traindf.loc[traindf['Contacted'] == 'N', 'Contacted_numeric'] = 2.0
traindf = traindf.drop('Contacted', axis=1)

#Test Data
testdf['Contacted_numeric']  = 0.0 # default value
testdf.loc[testdf['Contacted'] == 'Y', 'Contacted_numeric'] = 1.0
testdf.loc[testdf['Contacted'] == 'N', 'Contacted_numeric'] = 2.0
testdf = testdf.drop('Contacted', axis=1)

In [17]:
# Feature Engineering - Convert Categorical Data to Numeric > Source
# convert Source to numeric

#Train Data
traindf['Source_numeric']  = 0.0 # default value
traindf.loc[traindf['Source'] != 0.0, 'Source_numeric'] = traindf['Source'].str.slice(start=1).astype(float)
traindf = traindf.drop('Source', axis=1)

#Test Data

testdf['Source_numeric']  = 0.0 # default value
testdf.loc[testdf['Source'] != 0.0, 'Source_numeric'] = testdf['Source'].str.slice(start=1).astype(float)
testdf = testdf.drop('Source', axis=1)

In [18]:
# Feature Engineering - Convert Categorical Data to Numeric > Source_Category
# convert Source_Category to numeric
traindf['Source_Category'].unique()


#Train Data
traindf['Source_Category_numeric']  = 0.0 # default value
traindf.loc[traindf['Source_Category'] == 'A', 'Source_Category_numeric'] = 1.0
traindf.loc[traindf['Source_Category'] == 'B', 'Source_Category_numeric'] = 2.0
traindf.loc[traindf['Source_Category'] == 'C', 'Source_Category_numeric'] = 3.0
traindf.loc[traindf['Source_Category'] == 'D', 'Source_Category_numeric'] = 4.0
traindf.loc[traindf['Source_Category'] == 'E', 'Source_Category_numeric'] = 5.0
traindf.loc[traindf['Source_Category'] == 'F', 'Source_Category_numeric'] = 6.0
traindf.loc[traindf['Source_Category'] == 'G', 'Source_Category_numeric'] = 7.0
traindf = traindf.drop('Source_Category', axis=1)

#Test Data
testdf['Source_Category_numeric']  = 0.0 # default value
testdf.loc[testdf['Source_Category'] == 'A', 'Source_Category_numeric'] = 1.0
testdf.loc[testdf['Source_Category'] == 'B', 'Source_Category_numeric'] = 2.0
testdf.loc[testdf['Source_Category'] == 'C', 'Source_Category_numeric'] = 3.0
testdf.loc[testdf['Source_Category'] == 'D', 'Source_Category_numeric'] = 4.0
testdf.loc[testdf['Source_Category'] == 'E', 'Source_Category_numeric'] = 5.0
testdf.loc[testdf['Source_Category'] == 'F', 'Source_Category_numeric'] = 6.0
testdf.loc[testdf['Source_Category'] == 'G', 'Source_Category_numeric'] = 7.0
testdf = testdf.drop('Source_Category', axis=1)

In [19]:
# convert integer based columns to float
traindf['Var1'] = traindf['Var1'].astype(float)

In [20]:
# create a column 'Age' from (Lead_Creation_Date - DOB)

#Train Data
#print("Original LCD >>",traindf['Lead_Creation_Date'].unique())
#print("Original DOB >>",traindf['DOB'].unique())
#Fix DOB to start from 1900 prefix due to the way datetime works , less than 69 becomes 2068 etc. greater than 69 is 1969
traindf['DOB'] = traindf['DOB'].str.slice(stop=-2) + "19" + traindf['DOB'].str.slice(start=-2)

traindf['Lead_Creation_Date'] = pd.to_datetime(traindf['Lead_Creation_Date'], dayfirst=True)
#print("After datetime LCD >>",traindf['Lead_Creation_Date'].unique())
traindf['DOB'] = pd.to_datetime(traindf['DOB'], dayfirst=True)
#print("After datetime DOB >>", traindf['DOB'].unique())

traindf['Age_in_days'] = (traindf['Lead_Creation_Date']-traindf['DOB'])/timedelta(days=1)
#print("Age in Days >>",traindf['Age_in_days'].unique())
#print("Age in Days >>",traindf['Age_in_days' < 0].count())
#traindf['Age_in_days'] = traindf['Age_in_days'].abs()
traindf['Age_in_days'].fillna(traindf['Age_in_days'].dropna().mean(), inplace=True)
traindf = traindf.drop('Lead_Creation_Date', axis=1)
traindf = traindf.drop('DOB', axis=1)

#Test Data
#Fix DOB to start from 1900 prefix due to the way datetime works , less than 69 becomes 2068 etc. greater than 69 is 1969
testdf['DOB'] = testdf['DOB'].str.slice(stop=-2) + "19" + testdf['DOB'].str.slice(start=-2)
testdf['Lead_Creation_Date'] = pd.to_datetime(testdf['Lead_Creation_Date'])
testdf['DOB'] = pd.to_datetime(testdf['DOB'])
testdf['Age_in_days'] = (testdf['Lead_Creation_Date']-testdf['DOB'])/timedelta(days=1)
#testdf['Age_in_days'] = testdf['Age_in_days'].abs()
testdf['Age_in_days'].fillna(testdf['Age_in_days'].dropna().mean(), inplace=True)
testdf = testdf.drop('Lead_Creation_Date', axis=1)
testdf = testdf.drop('DOB', axis=1)


In [21]:
#list train data after feature engineering
traindf.groupby('Age_in_days').apply(lambda column: column < 0).sum()
#traindf.head(5)

Employer_Category2                             0
Monthly_Income                                 0
Existing_EMI                                   0
Loan_Amount                                    0
Loan_Period                                    0
Interest_Rate                                  0
EMI                                            0
Var1                                           0
Approved                                       0
Gender_numeric                                 0
City_Code_numeric                              0
City_Category_numeric                          0
Employer_Code_numeric                          0
Employer_Category1_numeric                     0
Customer_Existing_Primary_Bank_Code_numeric    0
Primary_Bank_Type_numeric                      0
Contacted_numeric                              0
Source_numeric                                 0
Source_Category_numeric                        0
Age_in_days                                    0
dtype: int64

In [22]:
testdf.head(10)

,Employer_Category2,Monthly_Income,Existing_EMI,Loan_Amount,Loan_Period,Interest_Rate,EMI,Var1,Gender_numeric,City_Code_numeric,City_Category_numeric,Employer_Code_numeric,Employer_Category1_numeric,Customer_Existing_Primary_Bank_Code_numeric,Primary_Bank_Type_numeric,Contacted_numeric,Source_numeric,Source_Category_numeric,Age_in_days
ID,,,,,,,,,,,,,,,,,,,
APPA70109647212,4.0,2150.0,0.0,10000.000000,3.000000,20.000000,372.000000,4,1.0,10028.0,3.0,2222.0,1.0,2.0,1.0,1.0,122.0,2.0,10289.0
APPB10687939341,4.0,4200.0,0.0,69000.000000,5.000000,24.000000,1985.000000,7,1.0,10003.0,1.0,1784.0,3.0,4.0,1.0,1.0,133.0,3.0,12596.0
APPC80449411414,4.0,1000.0,0.0,39482.990201,3.903116,19.280537,1094.914836,0,2.0,10009.0,2.0,45260.0,2.0,2.0,1.0,2.0,133.0,2.0,9180.0
APPD30665094501,3.0,1465.0,0.0,39482.990201,3.903116,19.280537,1094.914836,0,2.0,10005.0,1.0,85.0,1.0,1.0,1.0,2.0,133.0,3.0,8484.0
APPE80379821637,4.0,2340.0,500.0,10000.000000,2.000000,19.280537,1094.914836,0,1.0,10005.0,1.0,6422.0,1.0,4.0,1.0,1.0,143.0,2.0,9969.0
APPF90611935733,4.0,1500.0,450.0,39482.990201,3.903116,19.280537,1094.914836,0,2.0,10010.0,1.0,31460.0,2.0,4.0,1.0,2.0,133.0,2.0,10239.0
APPG20993415813,4.0,6900.0,3000.0,39482.990201,3.903116,19.280537,1094.914836,0,2.0,10002.0,1.0,1420.0,1.0,3.0,2.0,2.0,134.0,2.0,15447.0
APPH30656456927,4.0,2055.5,749.7,39482.990201,3.903116,19.280537,1094.914836,0,2.0,10003.0,1.0,1802.0,2.0,2.0,1.0,2.0,133.0,2.0,8847.0
APPI70297912043,4.0,3000.0,0.0,10000.000000,1.000000,19.280537,1094.914836,10,1.0,10014.0,2.0,35022.0,1.0,4.0,1.0,1.0,156.0,2.0,8979.0


In [23]:
# extract features from training set - all columns except 'Approved'
train_features = traindf.loc[:, traindf.columns != 'Approved']
train_features.columns

Index(['Employer_Category2', 'Monthly_Income', 'Existing_EMI', 'Loan_Amount',
       'Loan_Period', 'Interest_Rate', 'EMI', 'Var1', 'Gender_numeric',
       'City_Code_numeric', 'City_Category_numeric', 'Employer_Code_numeric',
       'Employer_Category1_numeric',
       'Customer_Existing_Primary_Bank_Code_numeric',
       'Primary_Bank_Type_numeric', 'Contacted_numeric', 'Source_numeric',
       'Source_Category_numeric', 'Age_in_days'],
      dtype='object')

In [24]:
# extract label from training set - Approved
train_label = traindf.loc[:, traindf.columns == 'Approved']
train_label.columns

Index(['Approved'], dtype='object')

In [25]:
# check for null valued columns
traindf.columns[traindf.isnull().any()].tolist()


[]

In [26]:
#for cols in train_features.columns.tolist()[1:]:
#    print(train_features.ix[train_features[cols] < 0])

In [27]:

'''from sklearn import feature_selection
fs = feature_selection.SelectPercentile(feature_selection.chi2, percentile=20)
train_features_fs = fs.fit_transform(train_features, train_label)
print('Scores of these features:', fs.scores_)
print(train_features_fs)
print(np.ravel(train_features_fs))
'''


"from sklearn import feature_selection\nfs = feature_selection.SelectPercentile(feature_selection.chi2, percentile=20)\ntrain_features_fs = fs.fit_transform(train_features, train_label)\nprint('Scores of these features:', fs.scores_)\nprint(train_features_fs)\nprint(np.ravel(train_features_fs))\n"

In [28]:

# define the parameter values that should be searched
n_range = list(range(1, 31))
min_samples_split_range  = list(range(2,10))
max_features_range = ['auto','sqrt','log2']

from sklearn.grid_search import RandomizedSearchCV
from sklearn.grid_search import GridSearchCV
# specify "parameter distributions" rather than a "parameter grid"
model = ExtraTreesClassifier(n_estimators=10, min_samples_split=5)
param_dict = dict(n_estimators=n_range, min_samples_split=min_samples_split_range, max_features=max_features_range)

conv_X = pd.get_dummies(train_features.iloc[:, :]) 
conv_Y = pd.get_dummies(train_label['Approved']) 
#print(conv_Y)
#print(conv_X)

# n_iter controls the number of searches
#rand = GridSearchCV(model, param_dict, cv=10, scoring='accuracy',  n_jobs=-1)
rand = RandomizedSearchCV(model, param_dict, cv=10, scoring='accuracy', n_iter=10, random_state=5)
rand.fit(conv_X, conv_Y)
print("GRID SCORES >>> ",rand.grid_scores_)

# examine the best model
print("BEST SCORE >>> ",rand.best_score_)
print("BEST PARAMETERS >>> ",rand.best_params_)


C:\ProgramData\Anaconda3-5.0.1\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3-5.0.1\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


GRID SCORES >>>  [mean: 0.98538, std: 0.00247, params: {'n_estimators': 18, 'min_samples_split': 9, 'max_features': 'log2'}, mean: 0.98534, std: 0.00246, params: {'n_estimators': 27, 'min_samples_split': 6, 'max_features': 'sqrt'}, mean: 0.96986, std: 0.00295, params: {'n_estimators': 1, 'min_samples_split': 4, 'max_features': 'sqrt'}, mean: 0.98517, std: 0.00247, params: {'n_estimators': 13, 'min_samples_split': 4, 'max_features': 'log2'}, mean: 0.98501, std: 0.00251, params: {'n_estimators': 24, 'min_samples_split': 2, 'max_features': 'auto'}, mean: 0.98535, std: 0.00249, params: {'n_estimators': 11, 'min_samples_split': 9, 'max_features': 'log2'}, mean: 0.98485, std: 0.00245, params: {'n_estimators': 4, 'min_samples_split': 6, 'max_features': 'auto'}, mean: 0.98533, std: 0.00249, params: {'n_estimators': 7, 'min_samples_split': 7, 'max_features': 'sqrt'}, mean: 0.98531, std: 0.00247, params: {'n_estimators': 14, 'min_samples_split': 5, 'max_features': 'sqrt'}, mean: 0.98530, std: 0.

In [34]:
#Train the model with best parameters of RF
# best params for RF using randomizedCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

model = make_pipeline(StandardScaler(with_std=True), PCA(n_components=8), ExtraTreesClassifier(n_estimators=18,min_samples_split=9))

model.fit(train_features, train_label)
train_pred = model.predict(train_features)

print(metrics.accuracy_score(train_label, train_pred)) # Training Accuracy Score
print (np.sqrt(mean_squared_error(train_label, train_pred))) # Training RMSE
print(roc_auc_score(train_label, train_pred)) # AUC-ROC values

C:\ProgramData\Anaconda3-5.0.1\lib\site-packages\sklearn\pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)


0.98558375052
0.120067687077
0.507352941176


In [35]:
#Predict with test data - predict probabilities
test_pred = model.predict_proba(testdf) #test features are all in testdf

print(test_pred) # Predicted Values
#print(np.unique(test_pred)) # unique values

[[ 0.99074074  0.00925926]
 [ 0.96111111  0.03888889]
 [ 1.          0.        ]
 ..., 
 [ 1.          0.        ]
 [ 0.99305556  0.00694444]
 [ 0.99305556  0.00694444]]


In [36]:
#Prepare outputdf to populate CSV
#output df
#print(test_pred.classes_)
#print(testdf.index)
#print(test_pred[:,1])
outputdf = pd.DataFrame()
outputdf['ID'] = testdf.index
outputdf['Approved'] = test_pred[:,1]

In [37]:
#outputdf

In [38]:
#Save to CSV file in submission format
outputdf.to_csv("output_etc.csv", sep=",", index=False)